In [1]:
import numpy as np
import pandas as pd

#Load the data
train_in = pd.read_csv('train_in - Copy.csv', header=None)
train_out = pd.read_csv('train_out - Copy.csv', header=None)
test_in = pd.read_csv('test_in - Copy.csv', header=None)
test_out = pd.read_csv('test_out - Copy.csv', header=None)

# Convert data to numpy arrays
train_in= train_in.to_numpy()
train_out = train_out.to_numpy().flatten()
test_in= test_in.to_numpy()
test_out = test_out.to_numpy().flatten()

In [2]:
#Seperate different numbers

Seperate_list_train=np.empty(10, dtype=object)
Seperate_list_train[:]=[[] for _ in range(10)]

for i in range(len(train_out)):
    Seperate_list_train[train_out[i]].append(train_in[i])


In [3]:
#Calculate the centers

Centers_train=np.empty(10, dtype=object)
for i in range(len(Seperate_list_train)):
    Centers_train[i]=np.mean(Seperate_list_train[i], axis=0)

In [4]:
#Calculate the distances

dist_train=np.zeros((10,10))
for i in range(10):
    for j in range(10):
        dist_train[i][j]=np.linalg.norm(Centers_train[i] - Centers_train[j])

print(dist_train)

[[ 0.         14.44960797  9.33455587  9.14373367 10.76984444  7.51929626
   8.15444313 11.86455505  9.90790174 11.48887494]
 [14.44960797  0.         10.12532258 11.7332329  10.17378643 11.11880041
  10.61470037 10.74315367 10.08677677  9.9320937 ]
 [ 9.33455587 10.12532258  0.          8.17828489  7.93254148  7.90679632
   7.33180754  8.87253107  7.07751618  8.88774785]
 [ 9.14373367 11.7332329   8.17828489  0.          9.0876078   6.11875002
   9.30206473  8.92240093  7.02042489  8.35435012]
 [10.76984444 10.17378643  7.93254148  9.0876078   0.          8.00151741
   8.78223265  7.58301228  7.38090899  6.01040793]
 [ 7.51929626 11.11880041  7.90679632  6.11875002  8.00151741  0.
   6.69869172  9.21195402  6.96738648  8.25853807]
 [ 8.15444313 10.61470037  7.33180754  9.30206473  8.78223265  6.69869172
   0.         10.8882374   8.58722228 10.44000352]
 [11.86455505 10.74315367  8.87253107  8.92240093  7.58301228  9.21195402
  10.8882374   0.          8.4677853   5.42647412]
 [ 9.907

In [5]:
#Find the closest pair

Closest_num_train=np.min(dist_train[np.nonzero(dist_train)])
Closest_pair_train=np.where(dist_train==Closest_num_train)

print(Closest_pair_train)

(array([7, 9]), array([9, 7]))


In [6]:
#Nearest mean classifier (the training set)
Nearest_train=[]
for i in range(len(train_in)):
    dis=np.zeros(10)
    for j in range(10):
       dis[j]=np.linalg.norm(Centers_train[j]-train_in[i])
    Nearest_train.append(np.where(dis==np.min(dis))[0][0])

con_ma_train=np.zeros((10,10))#The component (i,j) represents how many times i element is taken as j
for i in range(len(Nearest_train)):
    con_ma_train[train_out[i]][Nearest_train[i]]+=1
Overall_accuracy_train=np.trace(con_ma_train/len(train_in))
accuracy_train=con_ma_train.diagonal()/con_ma_train.sum(axis=1)

Least_acc_train=np.where(accuracy_train==np.min(accuracy_train))#most difficult to classify correctly for train set

acc_ma_train=con_ma_train/con_ma_train.sum(axis=1)
error_matrix_train=np.triu(acc_ma_train, k=1)+np.tril(acc_ma_train, k=-1).T#Sum up the (i,j) and (j,i) elements in accuracy_train (trace not included)
error_ratio_train=np.max(error_matrix_train)
error_pair_train=np.where(error_matrix_train==error_ratio_train)
print(con_ma_train)
print(Overall_accuracy_train)#total accuracy for the training set
print(accuracy_train)#accuracy in each digits for the training set
print(Least_acc_train)#most difficult to classify correctly
print(error_pair_train)#most common mistaken pair for Nearest mean classifier in the training set


[[271.   0.   0.   0.   2.   4.  36.   0.   6.   0.]
 [  0. 252.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  3.   0. 167.   9.   9.   1.   3.   4.   6.   0.]
 [  0.   0.   2. 120.   1.   3.   0.   1.   3.   1.]
 [  0.   8.   1.   0.  95.   0.   3.   0.   0.  15.]
 [  3.   0.   2.   3.   4.  67.   3.   1.   2.   3.]
 [ 10.   4.   5.   0.   2.   0. 129.   0.   1.   0.]
 [  0.   4.   0.   0.   2.   2.   0. 140.   1.  17.]
 [  1.   2.   1.  10.   2.   3.   1.   0. 121.   3.]
 [  0.   3.   0.   1.  10.   0.   0.   6.   0. 112.]]
0.8635032220269478
[0.84952978 1.         0.82673267 0.91603053 0.77868852 0.76136364
 0.85430464 0.84337349 0.84027778 0.84848485]
(array([5]),)
(array([0]), array([6]))


In [7]:
#Nearest mean classifier (the test set)
Nearest_test=[]
for i in range(len(test_in)):
    dis=np.zeros(10)
    for j in range(10):
       dis[j]=np.linalg.norm(Centers_train[j]-test_in[i])
    Nearest_test.append(np.where(dis==np.min(dis))[0][0])

con_ma_test=np.zeros((10,10))#The component (i,j) represents how many times i element is taken as j
for i in range(len(Nearest_test)):
    con_ma_test[test_out[i]][Nearest_test[i]]+=1
Overall_accuracy_test=np.trace(con_ma_test/len(test_in))
accuracy_test=con_ma_test.diagonal()/con_ma_test.sum(axis=1)

Least_acc_test=np.where(accuracy_test==np.min(accuracy_test))#most difficult to classify correctly for test set

acc_ma_test=con_ma_test/con_ma_test.sum(axis=1)
error_matrix_test=np.triu(acc_ma_train, k=1)+np.tril(acc_ma_train, k=-1).T#Sum up the (i,j) and (j,i) elements in accuracy_test (trace not included)
error_ratio_test=np.max(error_matrix_test)
error_pair_test=np.where(error_matrix_test==error_ratio_test)
print(con_ma_test)
print(Overall_accuracy_test)#total accuracy for the test set
print(accuracy_test)#accuracy in each digits for the test set
print(Least_acc_test)#most difficult to classify correctly
print(error_pair_test)#most common mistaken pair for Nearest mean classifier in the test set

[[178.   0.   3.   2.   4.   2.  23.   1.  10.   1.]
 [  0. 120.   0.   0.   0.   0.   1.   0.   0.   0.]
 [  2.   0.  69.   6.   8.   1.   0.   2.  13.   0.]
 [  3.   0.   3.  61.   1.   8.   0.   0.   1.   2.]
 [  1.   3.   3.   0.  69.   0.   1.   1.   0.   8.]
 [  3.   0.   0.   6.   3.  38.   1.   0.   0.   4.]
 [  7.   0.   2.   0.   2.   1.  78.   0.   0.   0.]
 [  0.   2.   1.   0.   5.   0.   0.  50.   0.   6.]
 [  3.   2.   0.   6.   3.   3.   0.   0.  73.   2.]
 [  0.   5.   0.   0.   8.   0.   0.   5.   2.  68.]]
0.804
[0.79464286 0.99173554 0.68316832 0.7721519  0.80232558 0.69090909
 0.86666667 0.78125    0.79347826 0.77272727]
(array([2]),)
(array([0]), array([6]))
